<a href="https://colab.research.google.com/github/nthanhkhang/Natural-Language-Processing/blob/main/OpenIE_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ensure that your java version >= 1.8

In [18]:
!java -version

openjdk version "11.0.10" 2021-01-19
OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04)
OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)


# Downloading Stanford CoreNLP (375mb, quite heavy) and unzip it

In [19]:
!wget http://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip

--2021-02-23 13:08:35--  http://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip [following]
--2021-02-23 13:08:35--  https://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 393239982 (375M) [application/zip]
Saving to: ‘stanford-corenlp-full-2018-10-05.zip.1’

stanford-corenlp-fu 100%[===================>] 375.02M  1.88MB/s    in 2m 7s   

2021-02-23 13:10:43 (2.95 MB/s) - ‘stanford-corenlp-full-2018-10-05.zip.1’ saved [393239982/393239982]



In [20]:
!unzip stanford-corenlp-full-2018-10-05.zip -d ./ && mv ./stanford-corenlp-full-2018-10-05 ./corenlp

Archive:  stanford-corenlp-full-2018-10-05.zip
   creating: ./stanford-corenlp-full-2018-10-05/
  inflating: ./stanford-corenlp-full-2018-10-05/jaxb-core-2.3.0.1-sources.jar  
  inflating: ./stanford-corenlp-full-2018-10-05/xom-1.2.10-src.jar  
  inflating: ./stanford-corenlp-full-2018-10-05/CoreNLP-to-HTML.xsl  
  inflating: ./stanford-corenlp-full-2018-10-05/README.txt  
  inflating: ./stanford-corenlp-full-2018-10-05/jollyday-0.4.9-sources.jar  
  inflating: ./stanford-corenlp-full-2018-10-05/LIBRARY-LICENSES  
   creating: ./stanford-corenlp-full-2018-10-05/sutime/
  inflating: ./stanford-corenlp-full-2018-10-05/sutime/british.sutime.txt  
  inflating: ./stanford-corenlp-full-2018-10-05/sutime/defs.sutime.txt  
  inflating: ./stanford-corenlp-full-2018-10-05/sutime/spanish.sutime.txt  
  inflating: ./stanford-corenlp-full-2018-10-05/sutime/english.sutime.txt  
  inflating: ./stanford-corenlp-full-2018-10-05/sutime/english.holidays.sutime.txt  
 extracting: ./stanford-corenlp-full-2

# Launch corenlp server in background

In [21]:
import os
os.system('cd ./corenlp/ && java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000 &')

0

# Test if it works (should return some json)

In [22]:
!curl --data 'The quick brown fox jumped over the lazy dog.' 'http://localhost:9000/?properties={%22annotators%22%3A%22tokenize%2Cssplit%2Cpos%22%2C%22outputFormat%22%3A%22json%22}' -o -

curl: (7) Failed to connect to localhost port 9000: Connection refused


# Installing Py-CoreNLP lib

In [23]:
!pip install pycorenlp

In [24]:
from pycorenlp import StanfordCoreNLP
from pprint import pprint

In [25]:
nlp = StanfordCoreNLP('http://corenlp.run/')

In [26]:
text = ('Pusheen and Smitha walked along the beach. '
        'Pusheen wanted to surf, but fell off the surfboard. '
        'The quick brown fox jumped over the lazy dog. '
        'Barack Obama was born in Hawaii.  '
        'He was elected president in 2008. ')
output = nlp.annotate(text, properties = {
  'annotators': 'tokenize,ssplit,pos,depparse,natlog,openie',
  'outputFormat': 'json'
})

In [27]:
for sentence in output['sentences']:
    print("="*50)
    print(f"Sentence: {' '.join([x['word'] for x in sentence['tokens']])}")
    print("-"*20)
    print("IE triplets:")
    print("-"*20)
    for triplet in sentence['openie']:
        print(f"{triplet['subject']:^20} -> {triplet['relation']:^14} -> {triplet['object']}")

Sentence: Pusheen and Smitha walked along the beach .
--------------------
IE triplets:
--------------------
      Pusheen        ->  walked along  -> beach
       Smitha        ->  walked along  -> beach
Sentence: Pusheen wanted to surf , but fell off the surfboard .
--------------------
IE triplets:
--------------------
      Pusheen        ->    fell off    -> surfboard
Sentence: The quick brown fox jumped over the lazy dog .
--------------------
IE triplets:
--------------------
  quick brown fox    ->  jumped over   -> lazy dog
        fox          ->  jumped over   -> dog
  quick brown fox    ->  jumped over   -> dog
     quick fox       ->  jumped over   -> dog
     brown fox       ->  jumped over   -> lazy dog
     brown fox       ->  jumped over   -> dog
     quick fox       ->  jumped over   -> lazy dog
        fox          ->  jumped over   -> lazy dog
Sentence: Barack Obama was born in Hawaii .
--------------------
IE triplets:
--------------------
    Barack Obama     ->  